In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

eSOH_DIR = "/Users/hamid/piibamm/PyBaMM/UMBLFeb2022/data/esoh/"
oCV_DIR = "/Users/hamid/piibamm/PyBaMM/UMBLFeb2022/data/ocv/"
fig_DIR = "../figures/figures_fit/"
res_DIR = "../data/results_fit/"
# %matplotlib widget

from scipy.io import savemat


In [2]:
## Added from old vrsion 


In [3]:
parameter_values = get_parameter_values()

In [4]:
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        # "loss of active material": ("stress-driven","none"),
        "loss of active material": "stress-driven",
        # "stress-induced diffusion": "true",
        "lithium plating": "irreversible",
    }
)
# spm.print_parameter_info()
param=spm.param

In [5]:
cell = 1

In [17]:
parameter_values.search("partial")

Inner SEI partial molar volume [m3.mol-1]	9.585e-05
Lithium metal partial molar volume [m3.mol-1]	1.3e-05
Negative electrode partial molar volume [m3.mol-1]	3.1e-06
Outer SEI partial molar volume [m3.mol-1]	9.585e-05
Positive electrode partial molar volume [m3.mol-1]	-7.28e-07


In [6]:
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spm,parameter_values)
print(Temp)

25


In [13]:
# pybamm.set_logging_level("WARNING")
pybamm.set_logging_level("NOTICE")

drive_cycle = pd.read_csv("F:\Stefanapolu application\CRC\Drive Cycle\US06_default_5Ah.csv", comment="#", header=None).to_numpy()
drive_cycle2 = pd.read_csv("F:\Stefanapolu application\CRC\Drive Cycle\US06_GM_5Ah.csv", comment="#", header=None).to_numpy()

experiment = pybamm.Experiment( # defult D.C. DRC-DRC *1.25
    [
        ("Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for .9 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 1.9 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06": drive_cycle},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment2 = pybamm.Experiment( #lab cycling
    [
        ("Discharge at "+c_rate_d+dis_set,
         "Rest for 5 min",
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50")
    ] *300,
    # ] *40,
    termination="50% capacity",
#     cccv_handling="ode",
)

## GM_DCRDCR
experiment3 = pybamm.Experiment( #GM_ DRC-DRC
    [
        ("Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for .6 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 1 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06_GM_5Ah": drive_cycle2},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment4 = pybamm.Experiment( #Gm-RCRC
    [
        ("Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for 1.6 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 3.1 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06_GM_5Ah": drive_cycle2},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment5 = pybamm.Experiment( # defult D.C. RC-RC
    [
        ("Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for 1.8 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 3.7 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06": drive_cycle},
    termination="50% capacity",
#     cccv_handling="ode",
)

In [14]:
parameter_values = get_parameter_values()
parameter_values.update(
    {
        "Positive electrode LAM constant proportional term [s-1]": 2.5112e-07,
        "Negative electrode LAM constant proportional term [s-1]": 2.4741e-06,
        "Positive electrode LAM constant exponential term": 1.1942,
        "Negative electrode LAM constant exponential term": 1.1942,
        "Negative electrode active material volume fraction": eps_n_data,
        "Positive electrode active material volume fraction": eps_p_data,
        "Initial temperature [K]": 273.15+Temp,
        "Ambient temperature [K]": 273.15+Temp,
        "SEI kinetic rate constant [m.s-1]": 1.08494281e-16,
        "EC diffusivity [m2.s-1]": 8.30909086e-19,
        "SEI growth activation energy [J.mol-1]": 1.58777981e+04,
        # "SEI kinetic rate constant [m.s-1]": 1.6827e-16,
        # "EC diffusivity [m2.s-1]": 2e-18,
        # "SEI growth activation energy [J.mol-1]": 0,
    },
    check_already_exists=False,
)


In [15]:
# all_sumvars_dict1 = cycle_adaptive_simulation(spm, parameter_values, experiment,SOC_0, save_at_cycles=1)

In [16]:
sim_long = pybamm.Simulation(spm, experiment=experiment2, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long = sim_long.solve(initial_soc=SOC_0, save_at_cycles=10  )

2023-03-09 11:37:44.524 - [NOTICE] callbacks.on_cycle_start(174): Cycle 1/300 (19.527 ms elapsed) --------------------
2023-03-09 11:37:44.526 - [NOTICE] callbacks.on_step_start(182): Cycle 1/300, step 1/4: Discharge at C/5 until 3V
2023-03-09 11:37:45.552 - [NOTICE] callbacks.on_step_start(182): Cycle 1/300, step 2/4: Rest for 5 min
2023-03-09 11:37:45.627 - [NOTICE] callbacks.on_step_start(182): Cycle 1/300, step 3/4: Charge at C/5 until 4.2V
2023-03-09 11:37:46.060 - [NOTICE] callbacks.on_step_start(182): Cycle 1/300, step 4/4: Hold at 4.2V until C/50
2023-03-09 11:37:46.858 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.861 Ah (originally 4.861 Ah, will stop at 2.430 Ah)
2023-03-09 11:37:46.859 - [NOTICE] callbacks.on_cycle_start(174): Cycle 2/300 (2.353 s elapsed) --------------------
2023-03-09 11:37:46.859 - [NOTICE] callbacks.on_step_start(182): Cycle 2/300, step 1/4: Discharge at C/5 until 3V
2023-03-09 11:37:47.101 - [NOTICE] callbacks.on_step_start(182): Cycle 2/3

2023-03-09 11:37:53.746 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.819 Ah (originally 4.861 Ah, will stop at 2.430 Ah)
2023-03-09 11:37:53.746 - [NOTICE] callbacks.on_cycle_start(174): Cycle 13/300 (9.247 s elapsed) --------------------
2023-03-09 11:37:53.746 - [NOTICE] callbacks.on_step_start(182): Cycle 13/300, step 1/4: Discharge at C/5 until 3V
2023-03-09 11:37:53.983 - [NOTICE] callbacks.on_step_start(182): Cycle 13/300, step 2/4: Rest for 5 min
2023-03-09 11:37:53.993 - [NOTICE] callbacks.on_step_start(182): Cycle 13/300, step 3/4: Charge at C/5 until 4.2V
2023-03-09 11:37:54.218 - [NOTICE] callbacks.on_step_start(182): Cycle 13/300, step 4/4: Hold at 4.2V until C/50
2023-03-09 11:37:54.350 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.815 Ah (originally 4.861 Ah, will stop at 2.430 Ah)
2023-03-09 11:37:54.350 - [NOTICE] callbacks.on_cycle_start(174): Cycle 14/300 (9.853 s elapsed) --------------------
2023-03-09 11:37:54.350 - [NOTICE] callbacks.on_st

2023-03-09 11:38:01.208 - [NOTICE] callbacks.on_step_start(182): Cycle 24/300, step 4/4: Hold at 4.2V until C/50
2023-03-09 11:38:01.361 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.771 Ah (originally 4.861 Ah, will stop at 2.430 Ah)
2023-03-09 11:38:01.362 - [NOTICE] callbacks.on_cycle_start(174): Cycle 25/300 (16.857 s elapsed) --------------------
2023-03-09 11:38:01.363 - [NOTICE] callbacks.on_step_start(182): Cycle 25/300, step 1/4: Discharge at C/5 until 3V
2023-03-09 11:38:01.595 - [NOTICE] callbacks.on_step_start(182): Cycle 25/300, step 2/4: Rest for 5 min
2023-03-09 11:38:01.608 - [NOTICE] callbacks.on_step_start(182): Cycle 25/300, step 3/4: Charge at C/5 until 4.2V
2023-03-09 11:38:01.880 - [NOTICE] callbacks.on_step_start(182): Cycle 25/300, step 4/4: Hold at 4.2V until C/50
2023-03-09 11:38:02.020 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.767 Ah (originally 4.861 Ah, will stop at 2.430 Ah)
2023-03-09 11:38:02.020 - [NOTICE] callbacks.on_cycle_

2023-03-09 11:38:08.458 - [NOTICE] callbacks.on_step_start(182): Cycle 36/300, step 3/4: Charge at C/5 until 4.2V
2023-03-09 11:38:08.696 - [NOTICE] callbacks.on_step_start(182): Cycle 36/300, step 4/4: Hold at 4.2V until C/50
2023-03-09 11:38:08.837 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.722 Ah (originally 4.861 Ah, will stop at 2.430 Ah)
2023-03-09 11:38:08.838 - [NOTICE] callbacks.on_cycle_start(174): Cycle 37/300 (24.333 s elapsed) --------------------
2023-03-09 11:38:08.839 - [NOTICE] callbacks.on_step_start(182): Cycle 37/300, step 1/4: Discharge at C/5 until 3V
2023-03-09 11:38:09.085 - [NOTICE] callbacks.on_step_start(182): Cycle 37/300, step 2/4: Rest for 5 min
2023-03-09 11:38:09.095 - [NOTICE] callbacks.on_step_start(182): Cycle 37/300, step 3/4: Charge at C/5 until 4.2V
2023-03-09 11:38:09.321 - [NOTICE] callbacks.on_step_start(182): Cycle 37/300, step 4/4: Hold at 4.2V until C/50
2023-03-09 11:38:09.455 - [NOTICE] callbacks.on_cycle_end(196): Capacity i

KeyboardInterrupt: 

In [11]:
qwrqwf

NameError: name 'qwrqwf' is not defined

In [ ]:
pybamm.plot_summary_variables(sol_long)

In [ ]:
sim_long.plot(
    [
#         "Negative particle surface concentration [mol.m-3]",
#         "Electrolyte concentration [mol.m-3]",
#         "Positive particle surface concentration [mol.m-3]",
        "Current [A]",
#         "Negative electrode potential [V]",
#         "Electrolyte potential [V]",
#         "Positive electrode potential [V]",
        "Terminal voltage [V]",
#         "X-averaged cell temperature",
        "Negative electrode SOC",
        "Positive electrode SOC",
#         "SEI thickness [m]",
        "Loss of capacity to SEI [A.h]",
        "Negative electrode capacity [A.h]",
        "Positive electrode capacity [A.h]",
        "Discharge capacity [A.h]"

        
    ]
)

In [ ]:
print(list(sol_long.summary_variables.keys()))

In [ ]:
type(sol_long.summary_variables[list(sol_long.summary_variables.keys())[1]])

In [ ]:
# mdic={}
# for i in range (1):
# # range(len(sol_long.summary_variables)):
# #     stringing=''.join([list(sol_long.summary_variables.keys())[i]])
#     mdic={f"{i}": sol_long.summary_variables[list(sol_long.summary_variables.keys())[i]]}
# savemat("summary",mdic)


In [17]:
ident=""

In [15]:
x100=sol_long.summary_variables["x_100"]
y100=sol_long.summary_variables["y_100"]
x0=sol_long.summary_variables["x_0"]
y0=sol_long.summary_variables["y_0"]
Cap=sol_long.summary_variables["Capacity [A.h]"]
LoC_SEI_summ=sol_long.summary_variables["Loss of capacity to SEI [A.h]"]
LAM_P=sol_long.summary_variables["Loss of active material in positive electrode [%]"]
LAM_N=sol_long.summary_variables["Loss of active material in negative electrode [%]"]
C_Neg=sol_long.summary_variables["C_n"]
C_pos=sol_long.summary_variables["C_p"]
LLI=sol_long.summary_variables["Loss of lithium inventory [%]"]
cycle_num=sol_long.summary_variables["Cycle number"]
t = sol_long["Time [s]"].entries
I = sol_long["Current [A]"].entries
Q = sol_long['Discharge capacity [A.h]'].entries
LoC_SEI = sol_long["Loss of capacity to SEI [A.h]"].entries
N_SoC = sol_long["Negative electrode SOC"].entries
P_SoC = sol_long["Positive electrode SOC"].entries

ECM_whole_res=sol_long["Local ECM resistance [Ohm]"].entries
ECM_sum_res=sol_long.summary_variables["Local ECM resistance [Ohm]"]
ECM_sum_res_change=sol_long.summary_variables['Change in local ECM resistance [Ohm]']
cycle_time=sol_long.summary_variables['Time [s]']

# exp = 30e6*sol_0["Cell thickness change [m]"].entries
# exp_n = 30e6*sol_0["Negative Electrode thickness change [m]"].entries
# exp_p = 30e6*sol_0["Positive Electrode thickness change [m]"].entries
# ee_p = sol_0["X-averaged positive electrode porosity"].entries
# ee_n = sol_0["X-averaged negative electrode porosity"].entries
# ee_s = sol_0["X-averaged separator porosity"].entries
# j_n = sol_0["X-averaged negative electrode total interfacial current density [A.m-2]"].entries
# j_p = sol_0["X-averaged positive electrode total interfacial current density [A.m-2]"].entries

# x = sol_0["x [m]"].entries
# c_e_0 = sol_0["Electrolyte concentration [mol.m-3]"].entries

In [18]:
mdic = {"x100"+ident: x100, "y100"+ident:y100, "x0"+ident:x0,"y0"+ident:y0,"Cap"+ident:Cap,
        "LoC_SEI_summ"+ident:LoC_SEI_summ,
        "LAM_P"+ident:LAM_P,"LAM_N"+ident:LAM_N,"C_Neg"+ident:C_Neg,"C_pos"+ident:C_pos,
        "LLI"+ident:LLI,"cycle_num"+ident:cycle_num,
        "t"+ident:t,"LoC_SEI"+ident:LoC_SEI,"N_SoC"+ident:N_SoC,"P_SoC"+ident:P_SoC,
        "Q"+ident:Q, "I"+ident:I,
        
        "ECM_whole_res":ECM_whole_res,"ECM_sum_res":ECM_sum_res,
        "ECM_sum_res_change":ECM_sum_res_change,"cycle_time":cycle_time 
       }
savemat("Outputs_Oct23_default_DRC_DRC",mdic)

In [ ]:
# savemat("x100_test", mdic)


In [ ]:
# print(x100)

In [ ]:
# import csv
# with open(r"F:\Git Hub rep\CRC\neew\PyBaMM\Untitled Folder\whatever.csv", 'w') as f:
#     # create the csv writer
#     writer = csv.writer(f)

# write a row to the csv file
# i=1
# for lines in sol_long.all_summary_variables:
# #     writer.writerow(words)
#     i=i+1
#     savemat(f'{i:02d}', lines)


In [ ]:
type(x100)

In [ ]:
sol_long.variables.search("charge")

In [ ]:
# plt.plot(sol_long["Time [s]"].entries[1:15133], sol_long["Current [A]"].entries[1:15133])

In [ ]:
# plt.plot(sol_long["Time [s]"].entries[1:15133], sol_long["Terminal voltage [V]"].entries[1:15133])

In [ ]:
 plt.plot(sol_long["Time [s]"].entries, 1-sol_long["Discharge capacity [A.h]"].entries/5)

In [ ]:
sol_long["Time [s]"].entries

In [ ]:
spm.variables.search("resistance")

In [13]:
sim_long.plot(["Local ECM resistance",
"Local ECM resistance [Ohm]",
               "Resistance [Ohm]"])


interactive(children=(FloatSlider(value=0.0, description='t', max=3945.6108050567304, step=39.4561080505673), …

In [ ]:
 plt.plot(sol_long["Time [s]"].entries, 1-sol_long["Discharge capacity [A.h]"].entries/5)

In [12]:
sol_long.summary_variables 

{'Minimum measured discharge capacity [A.h]': array([0.        , 0.00036851, 0.00073712, 0.00110602, 0.00147529,
        0.00184471, 0.00221447, 0.00258459, 0.00295499, 0.00332563,
        0.00369648, 0.00406769, 0.00443917, 0.00481101, 0.00518303,
        0.00555534, 0.00592799, 0.00630095, 0.00667415, 0.00704764,
        0.00742142, 0.0077955 , 0.00816991, 0.00854462, 0.00891964,
        0.00929488, 0.00967043, 0.0100462 , 0.01042226, 0.01079869,
        0.01117537, 0.01155231, 0.01192959, 0.01230719, 0.01268504,
        0.01306324, 0.01344171, 0.01382047, 0.01419957, 0.01457888,
        0.01495849, 0.01533846, 0.01571871, 0.01609915, 0.01647996,
        0.01686104, 0.01724238, 0.01762402, 0.01800603, 0.01838824,
        0.01877082, 0.01915368, 0.01953685, 0.01992041, 0.02030412,
        0.02068815, 0.02107244, 0.02145706, 0.02184199, 0.02222717,
        0.02261276, 0.02299862, 0.02338479, 0.02377123, 0.02415788,
        0.02454484, 0.02493207, 0.0253197 , 0.02570759, 0.02609573,
   